# Projeto 3 - Google Play Store Apps

Ciências dos Dados

**Arthur Ventrilho**

**Gabriel Zanetti**

**Roger Pina**

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import math
from scipy import stats
import mpmath
import matplotlib.pyplot as plt
import statsmodels.api as sm

C:\Users\Roger Pina\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# Função utilitária para fazer a regressão com constante adicionada
def regress(X,Y):
    X_cp = sm.add_constant(X)
    model = sm.OLS(Y,X_cp)
    results = model.fit()
    return results

In [3]:
#Função que transforma colunas categóricas em colunas quantitativas
def dummify(data, column_name):
    """
        Converte a coluna column_name em dummies / one-hot e as adiciona ao dataframe
        retorna uma copia do  df original *sem* a coluna que foi dummified
    """
    df = data.copy()
    # Nota: se você for um econometrista, pode se interessar por usar a opção dropFirst = True na get_dummies
    df2 = pd.concat([df.drop(column_name, axis=1), pd.get_dummies(data[column_name], prefix=column_name + "_")], axis=1)
    return df2

In [4]:
data = pd.read_csv('googleplaystore.csv')
data = data.dropna(axis=0)

In [5]:
del data['Genres']
del data['Last Updated']
del data['Current Ver']
del data['Android Ver']

In [6]:
#Tranformando Installs em quant
deletar = ['+', ',']
Installs_novo = []
for frase in data['Installs']:
    for c in deletar:
        frase = frase.replace(c , "")
    Installs_novo.append(frase)
Installs_novo
data['Installs'] = pd.Series((Installs_novo), index = data.index)

In [7]:
deletar = ['$']
Price_novo = []
for frase in data['Price']:
    for c in deletar:
        frase = frase.replace(c , "")
    Price_novo.append(frase)
data['Price'] = pd.Series((Price_novo), index = data.index)

In [8]:
data = data.drop(data[data.Size == "Varies with device"].index)
def retira_letras(coluna):
    listaA = []
    if coluna == 'Size':
        for linha in data[coluna]:
            if linha[-1] == 'k':
                listaA.append(int(float(linha[:-1]) * 1000))
            elif linha[-1] == 'M':
                listaA.append(int(float(linha[:-1]) * 1000000))
    data[coluna] = pd.Series((listaA), index = data.index)
retira_letras('Size')

In [9]:
data.head(5)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19000000,10000,Free,0,Everyone
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14000000,500000,Free,0,Everyone
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8700000,5000000,Free,0,Everyone
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25000000,50000000,Free,0,Teen
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2800000,100000,Free,0,Everyone


In [10]:
def transforma_em_numero():
    colunas_int = ['Reviews', 'Size', 'Installs']
    colunas_ft = ['Rating', 'Price']
    
    for coluna in colunas_ft:
        listaft = []
        for linha in data[coluna]:
            listaft.append(float(linha))
        data[coluna] = pd.Series((listaft), index = data.index)
    
    for coluna in colunas_int:
        lista = []
        for linha in data[coluna]:
            lista.append(int(linha))
        data[coluna] = pd.Series((lista), index = data.index)
transforma_em_numero()

In [11]:
#Declarando Categórico
data.Category = data.Category.astype('category')
data.Type = data.Type.astype('category')
data['Content Rating'] = data['Content Rating'].astype('category')

In [12]:
#Declarando Quantitativas
quantitativas = ['Rating', 'Reviews', 'Size', 'Installs', 'Price']

In [13]:
#Separando a coluna que será a saída
quant = quantitativas.copy()
quant.remove('Rating')

In [14]:
#Fazendo a regressão
results = regress(data.loc[:, quant], data.Rating)
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Rating   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     21.74
Date:                Thu, 08 Nov 2018   Prob (F-statistic):           7.33e-18
Time:                        10:01:40   Log-Likelihood:                -6221.9
No. Observations:                7723   AIC:                         1.245e+04
Df Residuals:                    7718   BIC:                         1.249e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.1328      0.009    476.509      0.000       4.116       4.150
Reviews     1.796e-08   4.31e-09      4.164      0.000     9.5e-09    2.64e-08
Size        1.577e-09   2.71e-10      5.819      0.000    1.05e-09    2.11e-09
Installs    3.215e-11   1.58e-10      0.204      0.838   -2.77e-10    3.41e-10
Price         -0.0006      0.000     -1.667      0.096      -0.001       0.000
==============================================================================
Omnibus:                     2752.071   Durbin-Watson:                   1.807
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            11889.252
Skew:                          -1.708   Prob(JB):                         0.00
Kurtosis:                       8.028   Cond. No.                     7.28e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.28e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [15]:
#Covertendo Category
cat = dummify(data, "Category")

In [16]:
quant.extend(['Category__ART_AND_DESIGN', 'Category__AUTO_AND_VEHICLES','Category__BEAUTY', 'Category__BOOKS_AND_REFERENCE',
       'Category__BUSINESS', 'Category__COMICS', 'Category__COMMUNICATION','Category__DATING', 'Category__EDUCATION', 'Category__ENTERTAINMENT',
       'Category__EVENTS', 'Category__FAMILY', 'Category__FINANCE','Category__FOOD_AND_DRINK', 'Category__GAME',
       'Category__HEALTH_AND_FITNESS', 'Category__HOUSE_AND_HOME','Category__LIBRARIES_AND_DEMO', 'Category__LIFESTYLE',
       'Category__MAPS_AND_NAVIGATION', 'Category__MEDICAL','Category__NEWS_AND_MAGAZINES', 'Category__PARENTING',
       'Category__PERSONALIZATION', 'Category__PHOTOGRAPHY','Category__PRODUCTIVITY', 'Category__SHOPPING', 'Category__SOCIAL',
       'Category__SPORTS', 'Category__TOOLS', 'Category__TRAVEL_AND_LOCAL','Category__VIDEO_PLAYERS', 'Category__WEATHER'])

In [17]:
#Covertendo Type
tp = dummify(cat, "Type")

In [18]:
quant.extend(['Type__Free','Type__Paid'])

In [19]:
#Covertendo Content Rating
df = dummify(tp, "Content Rating")

In [20]:
quant.extend(['Content Rating__Adults only 18+', 'Content Rating__Everyone','Content Rating__Everyone 10+', 'Content Rating__Mature 17+',
       'Content Rating__Teen', 'Content Rating__Unrated'])

In [21]:
results = regress(df.loc[:, quant], data.Rating)
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Rating   R-squared:                       0.040
Model:                            OLS   Adj. R-squared:                  0.035
Method:                 Least Squares   F-statistic:                     7.711
Date:                Thu, 08 Nov 2018   Prob (F-statistic):           4.38e-44
Time:                        10:01:40   Log-Likelihood:                -6105.7
No. Observations:                7723   AIC:                         1.230e+04
Df Residuals:                    7680   BIC:                         1.260e+04
Df Model:                          42                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const                               2.5278      0.065     38.742      0.000       2.400       2.656
Reviews                          1.753e-08   4.33e-09      4.051      0.000    9.05e-09     2.6e-08
Size                             1.077e-09   3.01e-10      3.582      0.000    4.87e-10    1.67e-09
Installs                         5.861e-11   1.57e-10      0.374      0.709   -2.49e-10    3.66e-10
Price                              -0.0009      0.000     -2.367      0.018      -0.002      -0.000
Category__ART_AND_DESIGN            0.2794      0.069      4.062      0.000       0.145       0.414
Category__AUTO_AND_VEHICLES         0.0424      0.066      0.641      0.522      -0.087       0.172
Category__BEAUTY                    0.1933      0.086      2.252      0.024       0.025       0.362
Category__BOOKS_AND_REFERENCE       0.2174      0.044      4.912      0.000       0.131       0.304
Category__BUSINESS                  0.0180      0.034      0.522      0.602      -0.050       0.086
Category__COMICS                    0.0228      0.075      0.303      0.762      -0.125       0.170
Category__COMMUNICATION            -0.0122      0.037     -0.329      0.742      -0.085       0.060
Category__DATING                   -0.1292      0.051     -2.513      0.012      -0.230      -0.028
Category__EDUCATION                 0.2785      0.050      5.527      0.000       0.180       0.377
Category__ENTERTAINMENT             0.0283      0.056      0.502      0.615      -0.082       0.139
Category__EVENTS                    0.3795      0.085      4.481      0.000       0.213       0.546
Category__FAMILY                    0.0630      0.016      3.856      0.000       0.031       0.095
Category__FINANCE                   0.0128      0.033      0.384      0.701      -0.053       0.078
Category__FOOD_AND_DRINK           -0.0129      0.057     -0.224      0.823      -0.125       0.100
Category__GAME                      0.1011      0.021      4.742      0.000       0.059       0.143
Category__HEALTH_AND_FITNESS        0.1115      0.036      3.095      0.002       0.041       0.182
Category__HOUSE_AND_HOME            0.0611      0.070      0.873      0.383      -0.076       0.198
Category__LIBRARIES_AND_DEMO        0.1096      0.067      1.631      0.103      -0.022       0.241
Category__LIFESTYLE                -0.0054      0.032     -0.165      0.869      -0.069       0.058
Category__MAPS_AND_NAVIGATION      -0.0913      0.054     -1.688      0.091      -0.197       0.015
Category__MEDICAL                   0.0614      0.031      2.003      0.045       0.001       0.122
Category__NEWS_AND_MAGAZINES        0.0412      0.042      0.992      0.321      -0.040       0.123
Category__PARENTING                 0.2407      0.079      3.054      0.002       0

In [22]:
del df['Installs']
results = regress(df.loc[:, quant], data.Rating)
results.summary()

C:\Users\Roger Pina\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  
C:\Users\Roger Pina\Anaconda3\lib\site-packages\pandas\core\indexing.py:1367: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


LinAlgError: SVD did not converge